In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
import datetime
import time

from scipy.stats import gaussian_kde

In [2]:
#Defining the size of the grid
n_cell_x = 10
n_cell_y = 10

def cells_around(center):
    cells = []
    for i in range(-1, 2):
        for j in range(-1, 2):
            cells.append(center + j * n_cell_x + i)
    return cells

def prepare_data(df, n_cell_x, n_cell_y):
    """
    Feature engineering and computation of the grid.
    """
    #Creating the grid
    size_x = 10. / n_cell_x
    size_y = 10. / n_cell_y
    eps = 0.00001  
    xs = np.where(df.x.values < eps, 0, df.x.values - eps)
    ys = np.where(df.y.values < eps, 0, df.y.values - eps)
    pos_x = (xs / size_x).astype(np.int)
    pos_y = (ys / size_y).astype(np.int)
    df['grid_cell'] = pos_y * n_cell_x + pos_x
    
    #Feature engineering
    fw = [500, 1000, 4, 3, 2, 10] #feature weights (black magic here)
    df.x = df.x.values * fw[0]
    df.y = df.y.values * fw[1]
    initial_date = np.datetime64('2014-01-01T01:01', dtype='datetime64[m]') 
    d_times = pd.DatetimeIndex(initial_date + np.timedelta64(int(mn), 'm') 
                               for mn in df.time.values)    
    df['hour'] = (d_times.hour + d_times.minute/60) * fw[2]
    df['weekday'] = d_times.weekday * fw[3]
    df['month'] = d_times.month * fw[4]
    df['year'] = (d_times.year - 2013) * fw[5]

    df = df.drop(['time'], axis=1) 
    return df
    

def process_one_cell(df_train, df_test, grid_id, th):
    """   
    Classification inside one grid cell.
    """   
    #Working on df_train
    df_cell_train = df_train.loc[df_train.grid_cell == grid_id] # [df_train.grid_cell.isin(cells_around(grid_id))]
    place_counts = df_cell_train.place_id.value_counts()
    mask = (place_counts[df_cell_train.place_id.values] >= th).values
    df_cell_train = df_cell_train.loc[mask]

    #Working on df_test
    df_cell_test = df_test.loc[df_test.grid_cell == grid_id]
    row_ids = df_cell_test.index
    
    test_accuracy = df_cell_test.accuracy.values
    
    #Preparing data
    le = LabelEncoder()
    y = le.fit_transform(df_cell_train.place_id.values)
    X = df_cell_train.drop(['place_id', 'grid_cell', 'accuracy'], axis=1).values.astype(int)
    X_test = df_cell_test.drop(['grid_cell', 'accuracy'], axis = 1).values.astype(int)
    
    #Applying the classifier
    clf = KNeighborsClassifier(n_neighbors=40, weights='distance', 
                               metric='manhattan', n_jobs = -1)
    clf.fit(X, y)
    y_pred = clf.predict_proba(X_test)
    for i, p in enumerate(np.swapaxes(y_pred, 0, 1)):
        df_placeacc = df_train[df_train['place_id'] == le.inverse_transform(i)]['accuracy'].values
        kde = gaussian_kde(df_placeacc)
        for j, q in enumerate(p):
            if (q != 0):
                y_pred[j][i] = y_pred[j][i] * kde(df_cell_test['accuracy'].values[j])
    
    pred_labels = le.inverse_transform(np.argsort(y_pred, axis=1)[:,::-1][:,:3])
    return pred_labels, row_ids

In [ ]:
print('Loading data ...')
df_train = pd.read_csv('train.csv',
                       usecols=['row_id','x','y','time','place_id','accuracy'], 
                       index_col = 0)
df_test = pd.read_csv('test.csv',
                      usecols=['row_id','x','y','time', 'accuracy'],
                      index_col = 0)

print('Preparing train data')
df_train = prepare_data(df_train, n_cell_x, n_cell_y)
print('Preparing test data')
df_test = prepare_data(df_test, n_cell_x, n_cell_y)

Loading data ...


In [4]:
start_time = time.time()

#Solving classification problems inside each grid cell
th = 5 #Keeping place_ids with more than th samples.
n_cells = n_cell_x*n_cell_y
preds = np.zeros((df_test.shape[0], 3), dtype=int)

for g_id in range(n_cells):
    if g_id % 1 == 0:
        print('iter: %s, time: %s sec' %(g_id, time.time() - start_time))
    
    #Applying classifier to one grid cell
    pred_labels, row_ids = process_one_cell(df_train, df_test, g_id, th)
    #Updating predictions
    preds[row_ids] = pred_labels

iter: 0, time: 0.0003437995910644531 sec
iter: 1, time: 193.6833381652832 sec
iter: 2, time: 422.3755044937134 sec
iter: 3, time: 672.4929497241974 sec
iter: 4, time: 905.3674488067627 sec
iter: 5, time: 1148.6600499153137 sec
iter: 6, time: 1406.3398413658142 sec
iter: 7, time: 1655.510540008545 sec


KeyboardInterrupt: 

In [ ]:
print('Generating submission file ...')
#Auxiliary dataframe with the 3 best predictions for each sample
df_aux = pd.DataFrame(preds, dtype=str, columns=['l1', 'l2', 'l3'])  
#Concatenating the 3 predictions for each sample
ds_sub = df_aux.l1.str.cat([df_aux.l2, df_aux.l3], sep=' ')
#Writting to csv
ds_sub.name = 'place_id'
ds_sub.to_csv('knn_submission_' + str(datetime.datetime.now().strftime("%Y-%m-%d-%H-%M")) + '.csv',
              index=True, header=True, index_label='row_id')         